# 网络中的网络（NiN）

前几节介绍的LeNet、AlexNet和VGG在设计上的共同之处是：先以由卷积层构成的模块充分抽取空间特征，再以由全连接层构成的模块来输出分类结果。其中，AlexNet和VGG对LeNet的改进主要在于如何对这两个模块加宽（增加通道数）和加深。本节我们介绍网络中的网络（NiN）[1]。它提出了另外一个思路，即串联多个由卷积层和“全连接”层构成的小网络来构建一个深层网络。


## NiN块

我们知道，卷积层的输入和输出通常是四维数组（样本，通道，高，宽），而全连接层的输入和输出则通常是二维数组（样本，特征）。如果想在全连接层后再接上卷积层，则需要将全连接层的输出变换为四维。回忆在[“多输入通道和多输出通道”](channels.ipynb)一节里介绍的$1\times 1$卷积层。它可以看成全连接层，其中空间维度（高和宽）上的每个元素相当于样本，通道相当于特征。因此，NiN使用$1\times 1$卷积层来替代全连接层，从而使空间信息能够自然传递到后面的层中去。图5.7对比了NiN同AlexNet和VGG等网络在结构上的主要区别。

![左图是AlexNet和VGG的网络结构局部，右图是NiN的网络结构局部](../img/nin.svg)

NiN块是NiN中的基础块。它由一个卷积层加两个充当全连接层的$1\times 1$卷积层串联而成。其中第一个卷积层的超参数可以自行设置，而第二和第三个卷积层的超参数一般是固定的。

In [0]:
import d2ltorch as d2lt
import torch
from torch import nn, optim
from torchsummary import summary

class NiNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, **kwargs):
        super(NiNBlock, self).__init__(**kwargs)
        self.nin = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=1),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.nin(x)

## NiN模型

NiN是在AlexNet问世不久后提出的。它们的卷积层设定有类似之处。NiN使用卷积窗口形状分别为$11\times 11$、$5\times 5$和$3\times 3$的卷积层，相应的输出通道数也与AlexNet中的一致。每个NiN块后接一个步幅为2、窗口形状为$3\times 3$的最大池化层。

除使用NiN块以外，NiN还有一个设计与AlexNet显著不同：NiN去掉了AlexNet最后的3个全连接层，取而代之地，NiN使用了输出通道数等于标签类别数的NiN块，然后使用全局平均池化层对每个通道中所有元素求平均并直接用于分类。这里的全局平均池化层即窗口形状等于输入空间维形状的平均池化层。NiN的这个设计的好处是可以显著减小模型参数尺寸，从而缓解过拟合。然而，该设计有时会造成获得有效模型的训练时间的增加。

In [0]:
class MyNiN(nn.Module):
    def __init__(self, in_channels, **kwargs):
        super(MyNiN, self).__init__(**kwargs)
        self.features = nn.Sequential(
            NiNBlock(in_channels, 96, kernel_size=11, stride=4, padding=0),
            nn.MaxPool2d(kernel_size=3, stride=2),
            NiNBlock(96, 256, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(kernel_size=3, stride=2),
            NiNBlock(256, 384, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2), nn.Dropout(0.5),
            # 标签类别数是10
            NiNBlock(384, 10, kernel_size=3, stride=1, padding=1),
        )
        
        # 全局平均池化层将窗口形状自动设置成输入的高和宽
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        # 将四维的输出转成二维的输出，其形状为(批量大小, 10)
        x = x.reshape(-1, 10)
        return x

我们构建一个数据样本来查看每一层的输出形状。

In [26]:
net = MyNiN(in_channels=1)
summary(net, (1, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          11,712
              ReLU-2           [-1, 96, 54, 54]               0
            Conv2d-3           [-1, 96, 54, 54]           9,312
              ReLU-4           [-1, 96, 54, 54]               0
            Conv2d-5           [-1, 96, 54, 54]           9,312
              ReLU-6           [-1, 96, 54, 54]               0
          NiNBlock-7           [-1, 96, 54, 54]               0
         MaxPool2d-8           [-1, 96, 26, 26]               0
            Conv2d-9          [-1, 256, 26, 26]         614,656
             ReLU-10          [-1, 256, 26, 26]               0
           Conv2d-11          [-1, 256, 26, 26]          65,792
             ReLU-12          [-1, 256, 26, 26]               0
           Conv2d-13          [-1, 256, 26, 26]          65,792
             ReLU-14          [-1, 256,

## 获取数据和训练模型

我们依然使用Fashion-MNIST数据集来训练模型。NiN的训练与AlexNet和VGG的类似，但这里使用的学习率更大。

In [27]:
root = '~/dataset'
lr, num_epochs, batch_size, device = 0.1, 5, 128, d2lt.try_gpu()

d2lt.params_init(net, init=nn.init.xavier_uniform_)
optimizer = optim.SGD(net.parameters(), lr=lr)
train_iter, test_iter = d2lt.load_data_fashion_mnist(root, batch_size, resize=224)
d2lt.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on gpu
epoch 1, loss 0.0163, train acc 0.212, test acc 0.216, time 65.4 sec
epoch 2, loss 0.0096, train acc 0.515, test acc 0.611, time 67.2 sec
epoch 3, loss 0.0066, train acc 0.679, test acc 0.729, time 69.3 sec
epoch 4, loss 0.0056, train acc 0.732, test acc 0.733, time 69.4 sec
epoch 5, loss 0.0046, train acc 0.780, test acc 0.796, time 69.3 sec


## 小结

* NiN重复使用由卷积层和代替全连接层的$1\times 1$卷积层构成的NiN块来构建深层网络。
* NiN去除了容易造成过拟合的全连接输出层，而是将其替换成输出通道数等于标签类别数的NiN块和全局平均池化层。
* NiN的以上设计思想影响了后面一系列卷积神经网络的设计。

## 练习

* 调节超参数，提高分类准确率。
* 为什么NiN块里要有两个$1\times 1$卷积层？去除其中的一个，观察并分析实验现象。




## 参考文献

[1] Lin, M., Chen, Q., & Yan, S. (2013). Network in network. arXiv preprint arXiv:1312.4400.

## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/1661)

![](../img/qr_nin.svg)